In [1]:
!pip install python-docx

In [80]:
import docx
from string import punctuation
import re
import csv  

In [81]:
from Bio import Entrez
email='simgeekiz48@gmail.com'
from Bio.Entrez import efetch, read

In [82]:
rba_path = 'data/1. Risk-benefit analysis Kinderformularium Oxycodone.docx'

In [83]:
csv_path = rba_path.replace('.docx', '.csv')


In [85]:
def search(query):
    Entrez.email = email
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='20',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = email
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results


In [86]:
# import the Document class 
# from the docx module
from docx import Document
  
# create an instance of a 
# word document we want to open
doc = Document(rba_path)

In [87]:
findBegin = False
referenceList = []

for i in range(0, len(doc.paragraphs)):
    if (doc.paragraphs[i].text.lower().startswith('references')):
        findBegin = True
    if (findBegin == True):
        referenceList.append(doc.paragraphs[i].text)

In [88]:
header = ['record_id', 'title', 'abstract', 'doi', 'final_included']
with open(csv_path, 'w', newline='') as csvfile:
    cw = csv.writer(csvfile, delimiter=',') #quotechar='|', quoting=csv.QUOTE_MINIMAL
    cw.writerow(header)

In [89]:
#Loop over the reference list in the docx file
for ref in referenceList:
    ref = ref.strip()
    if (ref.lower().startswith('reference')):
        continue
    elif (ref.strip() == ''):
        continue
    else:
        # Find the possible title
        titleIndices = [match.start() for match in re.finditer('"', ref)]
        textTitle  = ref[titleIndices[0]:titleIndices[1]].strip(punctuation).strip()
        print(textTitle)
        # Find the DOI
#         quer = ref.find('doi')
#         pre_search_query = ref[quer:]
#         search_query = pre_search_query.replace('doi:', '').strip(punctuation).strip()
        quer = re.findall(r'doi:?\s?(.+).?', ref)
        if len(quer) >= 2:
            print('Please check this: ', textTitle)
        else: 
            quer=quer[0]
        search_query = quer.strip(punctuation).strip()
        print(search_query)
        
        # Search the article in the Pubmed Database
        PubmedSearchResults = search(search_query)
        id_list = PubmedSearchResults['IdList']
        
        #Return the research result
        papers = fetch_details(id_list)
        for paperIndex, paper in enumerate(papers['PubmedArticle']):
            paperTitle = paper['MedlineCitation']['Article']['ArticleTitle'].strip(punctuation).strip()
            if 'Abstract' in paper['MedlineCitation']['Article']:
                abstract = ' '.join([str(x) for x in paper['MedlineCitation']['Article']['Abstract']['AbstractText']])
            else:
                abstract = ''
                print('has no abstract: ', id_list[paperIndex])
            if (paperTitle == textTitle):
#                 print('item found', paperTitle)
                doi = search_query
                data = [id_list[paperIndex], paperTitle, abstract, doi, 1]
                with open(csv_path, 'a') as csvfile:
                    cw = csv.writer(csvfile, delimiter=',')
                    cw.writerow(data)
            else:
                print('Something went wrong with this item: ', paperTitle )

Analgesics for the treatment of pain in children
10.1056/NEJMra012626
has no abstract:  12362012
Oxycodone versus codeine for triage pain in children with suspected forearm fracture: a randomized controlled trial
10.1097/PEC.0b013e3181850ca3
Controlled-release oxycodone for the management of pediatric postoperative pain
10.1016/j.jpainsymman.2003.08.007
Population pharmacokinetics of oxycodone in children 6 months to 7 years old
10.1177/0091270006286433
The management of pain in children with life-limiting illnesses
10.1016/j.pcl.2007.07.007
Comparison of oxycodone pharmacokinetics after buccal and sublingual administration in children
10.2165/00003088-200645070-00009
Pharmacokinetics of oxycodone after intravenous, buccal, intramuscular and gastric administration in children
10.2165/00003088-200443090-00004
Maturation of oxycodone pharmacokinetics in neonates and infants: Oxycodone and its metabolites in plasma and urine
10.1111/bcp.13164
Effectiveness of oxycodone, ibuprofen, or the 

### Alternatives

https://www.codegrepper.com/code-examples/python/how+to+read+docx+file+in+python
import docx2txt
my_text = docx2txt.process("test.docx")
print(my_text)

with open(rba_path, 'r', encoding='utf8') as f:
    lines = f.readlines()